<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background- padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Vehicle Routing and Delivery using STO(Script Table Operator)
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
This demo walks through a vehicle routing problem, we have a vehicle moving from point A to point B. Between these two points, there are several routes. The goal is to find the best of these routes. However, in real world scenario, we often would have not just one vehicle, but a fleet of vehicles and set locations.</p>

<p style = 'font-size:16px;font-family:Arial'>
The Problem Statement is to find optimal (lowest cost) routes for a fleet of trucks to deliver packages to their destinations.
The Questions that need to be answered are:
<li style = 'font-size:16px;font-family:Arial'>What combination of packages should be put on which truck? </li>
<li style = 'font-size:16px;font-family:Arial'>Is it better to fully load a truck or divide over more? </li>
<li style = 'font-size:16px;font-family:Arial'>How to plan the routes?</li>
</p>

<p style = 'font-size:16px;font-family:Arial'> In this demo we model this problem using a Mixed Integer Linear Programming (MILP) framework which has a linear objective function, linear constraints and a mixture of non-negative and binary decision variables. The problem is formulated in Python using pulp library and cbc solver was used to solve it. We use the Script operator from teradataml to use STO, the parameters are set according to requirements. The output of the STO is used for further analysis such as solution validation and network diagrams. </p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to the Vantage.</b></p>


<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
!pip install pulp==2.4.0

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
import json
import getpass
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
# from pulp import *
import teradatasql
from teradataml.context.context import *
from collections import OrderedDict,defaultdict
from teradatasqlalchemy import (VARCHAR, INTEGER, FLOAT)
from teradataml.table_operators.Script import *
from teradataml import *
from teradataml.options.display import display
import networkx as nx
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore", category=DeprecationWarning)
# display.print_sqlmr_query = True
display.max_rows=5

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Vehicle_Routing_Delivery_STO_PY_SQL.ipynb;' UPDATE FOR SESSION; ''')

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
display.print_sqlmr_query = True

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>In this demo we are generating synthetic data. The data generated contains data for Trucks and Packages.</p>   


<p style = 'font-size:16px;font-family:Arial'>In the first step we randomly generate data of 10 trucks with volume capacity ranging from 5.7m${^3}$ to 9.8m${^3}$ (typical delivery truck size) .We are using the random function to get the volume and weight for each truck.</p>

In [ ]:
n = 10 # n trucks
np.random.seed(1)

# truck max volumes in cm3 - converted from m^3
truck_vol = np.random.randint(5.7*1000000,9.8*1000000, n) 

truck_ids = np.arange(1, n+1)

# truck weight capacities in lbs
max_wt_lbs = np.random.randint(1980, 2640, n) 

stack = np.stack([truck_ids, truck_vol, max_wt_lbs], axis=1)
df = pd.DataFrame.from_records(stack)
df.columns = ['truck_id', 'max_vol_cm3', 'max_wt_lbs']

<p style = 'font-size:16px;font-family:Arial'>We store this generated data for Trucks in a table in Vantage.</p>

In [ ]:
copy_to_sql(df, table_name='TRUCKS', if_exists = 'replace') # push to Vantage

In [ ]:
query = '''sel cast(truck_id as int) as truck_id,cast(max_vol_cm3 as int) as max_vol_cm3 ,cast(max_wt_lbs as int) 
as max_wt_lbs from trucks;'''

truck_df = pd.read_sql_query(query, eng)
truck_df

<p style = 'font-size:16px;font-family:Arial'>In the next step we generate data for packages.</p>

<p style = 'font-size:16px;font-family:Arial'>First we create different zones where the packages are to be delivered.</p>

In [ ]:
# Number of packages to be delivered in each zone
zones = np.concatenate([np.repeat('A', 15), np.repeat('B', 20), 
                        np.repeat('C', 25), np.repeat('D', 20),
                        np.repeat('E', 10), np.repeat('F', 15), 
                        np.repeat('G', 20), np.repeat('H', 25),
                        np.repeat('J', 20), np.repeat('T', 10)])
np.random.shuffle(zones)
zones

<p style = 'font-size:16px;font-family:Arial'>Then we create packages with different volumes using the random function and then assign the zones created above to each of these packages.</p>

In [ ]:
np.random.seed(1)
p = len(zones) # p packages
pkg_id = np.arange(1, p+1)
pkg_zone = zones

# Typical 2kg pkg max dims are 90x60x60 = 324000.
# Assume enough space for all pkgs, hence 200k mean
pkg_vol = np.random.normal(200000, 25000, p).astype(int)

pkg_stack = np.stack([pkg_id, pkg_zone, pkg_vol], axis=1)
pdf = pd.DataFrame.from_records(pkg_stack)
pdf.columns = ['pkg_id', 'pkg_zone', 'pkg_vol_cm3']

In [ ]:
pdf['pkg_id'] = pdf['pkg_id'].astype(int)
pdf['pkg_vol_cm3'] = pdf['pkg_vol_cm3'].astype(int)
pdf['pkg_zone'] = pdf['pkg_zone'].astype(str)

In [ ]:
pkg_df = pdf.copy()
pkg_df

In [ ]:
copy_to_sql(pkg_df, table_name= 'PACKAGES', if_exists = 'replace')
pkg_df =pd.read_sql_query('SEL * FROM PACKAGES;', eng)
pkg_df

<p style = 'font-size:16px;font-family:Arial'>In the next step we will create links between the Zone and assign a cost associated with the transport from one zone to other.</p>
<p style = 'font-size:16px;font-family:Arial'>We create a table for storing this data.</p>

In [ ]:
query = '''CREATE Multiset TABLE zone_links (
link_id INTEGER,
node_i VARCHAR(5),
node_j VARCHAR(5),
cost INTEGER
);'''

try:
    execute_sql(query)

except:
    db_drop_table('zone_links')
    execute_sql(query)

In [ ]:
query = '''
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (1,'O','A',2);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (2,'O','B',5);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (3,'O','C',4);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (4,'A','B',2);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (5,'A','D',7);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (6,'B','C',1);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (7,'B','D',4);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (8,'B','E',3);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (9,'C','E',4);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (10,'D','E',1);

INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (12,'E','F',1);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (13,'E','G',2);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (14,'E','H',3);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (15,'E','J',4);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (16,'D','F',5);

INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (17,'F','T',4);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (18,'H','T',5);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (19,'D','T',5);
INSERT INTO zone_links (link_id, node_i, node_j, cost) VALUES (20,'E','T',7);'''

execute_sql(query)

In [ ]:
z = pd.read_sql_query("SEL * FROM zone_links order by link_id", eng)
z

<p style = 'font-size:16px;font-family:Arial'>In the next step we will concatenate all the 3 tables to send to the STO.</p>

In [ ]:
#STO can only be given one table as input, so concatenating all tables
df_final = pd.concat([truck_df,pkg_df, z],axis=1,sort=False)
df_final

In [ ]:
copy_to_sql(df_final, table_name='actual_table', index = False, if_exists = 'replace')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Create the best route for trucks using Mixed Integer Linear Programming (MLIP) in python.<b style = 'font-size:14px;font-family:Arial'>(MILP_continuous.py).</b></p>
    
<p style = 'font-size:16px;font-family:Arial'>The  python file take table as input which includes information about packages , truck and the network. These values are used to set up a mixed integer linear programming (MILP) problem in pulp library. The objective function and constraints are defined in pulp as well. The problem is solved and the result is returned as a table.<p>
    
<p style = 'font-size:16px;font-family:Arial'>Below is the explanation for the python code which will be executed using STO. In the python file we first create a binary matrix of the packages and Zones</p>    

<p style = 'font-size:16px;font-family:Arial'><b>Create matrix of packages and zones</b></p>

```python 
#Delivery binary matrix of package and zones

D = pkg_df.copy()

D = pd.get_dummies(data=D, columns=['pkg_id', 'pkg_zone']) #.astype(int) # .set_index('pkg_id')

```  

<p style = 'font-size:16px;font-family:Arial'>We create the zone links and nodes to connect the zones</p>    

```python 
#Delivery binary matrix of package and zones
z2 = pd.DataFrame()
# create reverse links for two-way traversing and append
z2['node_i'] = z['node_j'] 
z2['node_j'] = z['node_i']
z2['cost'] = z['cost']
z2['link_id'] = np.arange(len(z)+1, len(z)+len(z)+1)

zone_links = z.append(z2).reset_index(drop=True)
zone_links.sort_values(by='link_id')

#Nodes that require deliveries
nodes = list(np.unique(zone_links.node_i.tolist() + zone_links.node_j.tolist()))
```

<p style = 'font-size:16px;font-family:Arial'><b>Assign packages to Trucks and create data for each truck.</b></p>

```python
# packages and trucks
t_ids = truck_df['truck_id'].tolist()
p_ids = pkg_df['pkg_id'].tolist()

pkg_vols = dict(zip(pkg_df['pkg_id'].tolist(), pkg_df['pkg_vol_cm3'].tolist()))
truck_vols = dict(zip(truck_df['truck_id'].tolist(), truck_df['max_vol_cm3'].tolist()))
pt_assign = [(p,t) for p in p_ids for t in t_ids]

# Create data for each truck
for t in t_ids:
    exec(f'zone_links_t{t} = zone_links.copy()')
    exec(f"zone_links_t{t}['node_i'] = zone_links['node_i']+'{t}'")
    exec(f"zone_links_t{t}['node_j'] = zone_links['node_j']+'{t}'")
    
    exec(f"node_i{t} = zone_links_t{t}['node_i'].tolist()")
    exec(f"node_j{t} = zone_links_t{t}['node_j'].tolist()")
    exec(f"arcs_t{t} = list(zip(node_i{t}, node_j{t}))")
    
    exec(f"arcData_t{t} = dict(zip(arcs_t{t}, (zone_links_t{t}.loc[i,['cost']].tolist() for i in zone_links_t{t}.index)))")
    exec(f"costs_t{t} = splitDict(arcData_t{t})[0]")
```

<p style = 'font-size:16px;font-family:Arial'><b>Create the LpVariable using the package and the truck assignment data and then create the LpProblem variable.</b></p>



```python
# pkg and truck assignment variable
pt_assign_var = LpVariable.dicts("Assign pkg truck", pt_assign, cat=LpBinary, lowBound=0, upBound=1)
    
for t in t_ids:
    # assignment var pkg-truck
    exec(f"P{t} = [pt_assign_var[(i, {t})] for i in p_ids]") 
    # Flow at Origin - Production nodes - sum of all pkgs assigned to a truck
    exec(f"B{t} = lpSum(P{t})")
    # Creates the selector/binary variables on each segment for each truck
    exec(f"selector_vars_t{t} = LpVariable.dicts(\"Route_select_t{t}\", arcs_t{t}, cat=LpBinary, lowBound=0)")
    # Z vars 
    exec(f"Z_var_t{t} = LpVariable.dicts(\"Z_{t}\", arcs_t{t}, cat=LpInteger, lowBound=0)")
    # Consumption variables 
    for n in nodes:
        exec(f"F{n}{t} = lpSum((D['pkg_zone_{n}'] * P{t}))*-1") # Diz * Pij 
        
# Create total demand at nodes variable 
F = defaultdict(str)
for n in nodes:
    for t in t_ids:
        exec(f"a = F{n}{t}")
        F[n, t] += a
        
# Creates the 'prob' variable to contain the problem data 
prob = LpProblem("Min Cost Flow Problem", LpMinimize)
```

<p style = 'font-size:16px;font-family:Arial'><b>Each package is than assigned to a truck based on the volume of the package and the volume of the truck.</b></p>



```python
# Each package needs to be assigned to only one truck of all
for i in p_ids:
    prob += lpSum([pt_assign_var[(i,t)] for t in t_ids]) == 1

M = 100
# print(prob.variables())

for t in t_ids:
    # Volumes of all pkgs on a truck <= Vol of truck
    prob += lpSum([pt_assign_var[(i,t)] * pkg_vols[i] for i in p_ids]) <= truck_vols[t] 
```    

<p style = 'font-size:16px;font-family:Arial'><b>The objective function of the LpProblem is created and is solved using the Pulp's choice of solver.</b></p>



```python
# Creates the objective function

# Flows and costs of all trucks
prob += lpSum([flows_dict[a] * costs_dict[a] for a in arcs_list]), "Total Cost of Transport"

# The problem is solved using PuLP's choice of Solver
LpSolverDefault.msg = 0
prob.solve()
```

<p style = 'font-size:16px;font-family:Arial'>Finally, the output which is the variable and the cost assigned to the variable are passed as output of the script which is returned to the STO function as a table</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>4. Using Script Command to get the forecasted values back to Vantage.</b></p>

<p style = 'font-size:16px;font-family:Arial'>The SCRIPT COMMAND requires the below elements 
<li style = 'font-size:16px;font-family:Arial'><b>ON clause: </b> The SCRIPT function can have only one ON clause (single input). The ON clause can be specified with no options or with: HASH BY, PARTITION BY, PARTITION BY ANY, an optional ORDER BY or LOCAL ORDER BY clause</li>
<li style = 'font-size:16px;font-family:Arial'><b>SCRIPT_COMMAND: </b>
The script to be executed. The SCRIPT_COMMAND is a required keyword.
    <li style = 'font-size:16px;font-family:Arial'><b>runtime_literal_command: </b>The parameters to SCRIPT_COMMAND can be an executable name followed by the script name and other inputs, or any valid LINUX command.</li>
<li style = 'font-size:16px;font-family:Arial'><b>RETURNS: </b>
    The names and types of the output columns returned by the script. <b>* </b>Specifies that all columns of the input table should be returned by the SCRIPT function.</li>
    
<p style = 'font-size:16px;font-family:Arial'>First we will create a dataset which can be passed to the Script function.</p>    

In [ ]:
final_table_df2 = DataFrame.from_query('''SELECT 1 as v_id, cast(case when truck_id IS NULL then 0 else truck_id end as Int) as truck_id,
case when max_vol_cm3 IS NULL then 0 else max_vol_cm3 end as max_vol_cm3, 
case when max_wt_lbs IS NULL then 0 else max_wt_lbs end as max_wt_lbs,pkg_id,pkg_zone,pkg_vol_cm3,
cast(case when link_id IS NULL then 0 else link_id end as int) as link_id ,
case when node_i IS NULL then 'None' else node_i end as node_i ,
case when node_j IS NULL then 'None' else node_j end as node_j ,
case when cost IS NULL then 0 else cost end as cost FROM actual_table;''')
final_table_df2

<p style = 'font-size:16px;font-family:Arial'>First we set the Database for execution to the user database: demo_user in this case</p>

In [ ]:
database_name = 'demo_user'

<p style = 'font-size:16px;font-family:Arial'>Set the search path to the database where the file is installed</p>

In [ ]:
execute_sql(f"SET SESSION SEARCHUIFDBPATH = {database_name};")

<p style = 'font-size:16px;font-family:Arial'>Install the python file for sto execution.</p>

<p style = 'font-size:16px;font-family:Arial'>Create the STO object by passing the dataframe with required data, the python file, the delimiter, the return variables along with the partition and order columns.</p>

In [ ]:
sto = Script(data = final_table_df2, script_name='MILP_continuous.py',
             files_local_path= r".",
             script_command= f'tdpython3 ./{database_name}/MILP_continuous.py', 
                         delimiter = "\t", nulls_first = False,
                         returns={"Variable":VARCHAR(50), "Value1": VARCHAR(50)}, # }, #,  "Variable":VARCHAR(50)
                         data_order_column = "pkg_id", #charset='latin',
                        data_partition_column="v_id",
                         is_local_order = False, sort_ascending=False)

try:
    sto.install_file(file_identifier='MILP_continuous', file_name='MILP_continuous.py', is_binary=False)
except:
    sto.remove_file(file_identifier='MILP_continuous', force_remove=False)
    sto.install_file(file_identifier='MILP_continuous', file_name='MILP_continuous.py', is_binary=False)

<p style = 'font-size:16px;font-family:Arial'>Execute the sto command to get the output in dataframe.</p>

In [ ]:
DF = sto.execute_script()
DF

In [ ]:
df = DF.to_pandas()
df = df[~df.Value1.str.contains('None')]
df['Value1'] = df['Value1'].astype(float)

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>5. Validation of package deliveries.</b></p>

<p style = 'font-size:16px;font-family:Arial'>After the STO script has executed and returned the table, we do additional verifications and validations on the solution. We verify whether:
<li style = 'font-size:16px;font-family:Arial'>All packages are delivered</li>
<li style = 'font-size:16px;font-family:Arial'>The packages are delivered to their correct destination</li>
<li style = 'font-size:16px;font-family:Arial'>The truck takes only one out of all possible route at each node</li>
<li style = 'font-size:16px;font-family:Arial'>The total cost of solution is computed manually and matched</li>

<p style = 'font-size:16px;font-family:Arial'> Assignment validation :-  Check if all packages were assigned to trucks </p>

In [ ]:
assignments = []
df_assignments =df[df['Variable'].str.contains('Assign')& df['Value1']>0]
#print(df_assignments.head())
#print(re.findall('\d+',df_assignments['Variable'].str))
df_assignments[['Pkg','Truck']] = df_assignments.Variable.astype('str').str.extractall('(\d+)').unstack().astype(int)
df_assignments


In [ ]:
for t in np.unique(df_assignments['Truck']):
    exec(f"Ps_on_T{t} = df_assignments[df_assignments.Truck=={t}].Pkg.to_list()")

In [ ]:
df_assignments[['Truck', 'Pkg']].groupby('Truck').count() 

<p style = 'font-size:16px;font-family:Arial'>Flows Validation :- Check if the trucks take the shortest route </p>

In [ ]:
df_flows =df[df['Variable'].str.contains('Z')& df['Value1']!=0]
df_flows.reset_index(drop=True,inplace = True)
df_flows

In [ ]:
df_edges =df[df['Variable'].str.contains('select')& df['Value1']!=0]
df_edges.reset_index(drop=True,inplace = True)
df_edges

<p style = 'font-size:16px;font-family:Arial'>Total Cost :- Calculate the total cost manually and validate </p>

In [ ]:

#Create z2 from z with node directions inverted

z2 = pd.DataFrame()
# create reverse links and append
z2['node_i'] = z['node_j'] 
z2['node_j'] = z['node_i']
z2['cost'] = z['cost']
z2['link_id'] = np.arange(len(z)+1, len(z)+len(z)+1)
zone_links = pd.concat([z,z2],axis=0,sort=False)
#zone_links


total_cost = 0
df_flows['key1'] = df_flows.Variable.str.split("'").str[1].str[0]

df_flows['key2'] = df_flows.Variable.str.split("'").str[3].str[0]
df_flows['cost'] = 0

merged_df = df_flows.merge(zone_links, left_on=['key1', 'key2'], right_on=['node_i', 'node_j'], how='left')
merged_df
df_flows['cost'] = merged_df['cost_y']
df_flows


df_flows['total_cost'] = df_flows['cost'].multiply(df_flows['Value1'])
print('The total cost is', df_flows['total_cost'].sum())

<p style = 'font-size:16px;font-family:Arial'>Delivery and Demand validation :- Check if all packages tagged were delivered to the actual location.</p>

In [ ]:
nodes = list(np.unique(zone_links.node_i.tolist() + zone_links.node_j.tolist()))
nodes.remove("O")
nodes

In [ ]:
# Pkg ids belonging to given nodes as per original tagging

D = pkg_df.copy()
D = pd.get_dummies(D[['pkg_id', 'pkg_zone']]).set_index('pkg_id').astype(int)
demands = dict(list(zip(sorted(nodes), D.sum())))
for n in nodes:
    exec(f"{n}_ids = list(D[D['pkg_zone_{n}']==1].index)")

In [ ]:
# Check if all pkgs orignially tagged for each node were actually delivered at the node

# Dict that contains list of all package ids delivered to a node
pkgs_delivered = defaultdict(list)
t_ids = truck_df['truck_id'].to_list()
for n in nodes:
    # original tagging
    original_tag = set(D[D[f'pkg_zone_{n}']==1][f'pkg_zone_{n}'].index)
    for t in t_ids:
        # packages on truck
        exec(f"pkgs_on_truck = set(Ps_on_T{t})")
        # append pkg ids delivered to a node by a truck and was supposed to be
        pkgs_delivered[n] += list(original_tag.intersection(pkgs_on_truck))
        
    exec(f"print(n, sorted(pkgs_delivered[n]) == {n}_ids)")

In [ ]:
df_flows

<p style = 'font-size:16px;font-family:Arial'>Demand check :- Check if the Demand and the Net flow is the same</p>

In [ ]:
# Demand meeting check


df_flows
for n in nodes:
    
    
    inflow = df_flows[df_flows['key2'] ==n]['Value1'].sum().astype(int)
    outflow = df_flows[df_flows['key1'] ==n]['Value1'].sum().astype(int)
    assert(demands[n] <= inflow-outflow)
    print(f'{n}:', 'Demand:', demands[n], f'\n{n}: Net flow:', inflow-outflow)
    assert(demands[n] <= inflow-outflow)


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>6. Visualization of the routes.</b></p>

<p style = 'font-size:16px;font-family:Arial'> We create a python function to visualize the routes using the network diagram</p>

In [ ]:
df_flows_new =df[df['Variable'].str.contains('Z')]
df_flows_new.reset_index(drop=True,inplace = True)
def graph_viz(t):
    # Create an empty directed graph
    G = nx.DiGraph()
    
   
    
    temp_nodes = [node + f'{t}' for node in nodes]
    exec(f"graph_nodes_t{t} = temp_nodes") #['A1', 'B1', 'C1', 'D1', 'E1', 'O1', 'T1']
    #exec(f"graph_nodes_t{t} = ['A1', 'B1', 'C1', 'D1', 'E1', 'O1', 'T1']")

    exec(f"G.add_nodes_from(graph_nodes_t{t})")

    # Define the edge labels as a dictionary
    edgeLabels = defaultdict(tuple)
    for i in range(df_flows_new.shape[0]):
        
        if(int(df_flows_new.loc[i,'Variable'].split('_')[1]) == t):
            
        
            #print(df_flows_new.loc[i,'Variable'].split("'")[1],df_flows_new.loc[i,'Variable'].split("'")[3],df_flows_new.loc[i,'Value1'])
            edgeLabels[(df_flows_new.loc[i,'Variable'].split("'")[1],df_flows_new.loc[i,'Variable'].split("'")[3])] = df_flows_new.loc[i,'Value1']
#     for v in prob.variables():
#         if f'Z_{t}' in v.name:
#             edgeLabels[(v.name.split("'")[1], v.name.split("'")[3])] = v.varValue
#         else:
#             pass


    # Create a set to store the filtered pairs
    filtered_pairs = set()
    

    # If a nonzero pair exists, the reverse-order pair with zero flow needs to be removed 
    # to avoid overwriting the nonzero flow in edge labels
    for pair, value in edgeLabels.items():
        reverse_pair = (pair[1], pair[0]) 
        reverse_value = edgeLabels.get(reverse_pair)
        #filtered_pairs.add(pair)
        

        if (value != 0.0 and reverse_value == 0.0) or (value == 0.0 and reverse_value==0.0):
            #print('hello ji')
            filtered_pairs.add(pair)
            
        else:
            pass


    # update edge labels
    edge_labels = {key: edgeLabels[key] for key in list(filtered_pairs)}
#     print(edge_labels)

    # Add edges to the graph and assign the weights using the edge_labels dictionary
    G.add_edges_from(edge_labels.keys())

    # Set node color
    node_color = 'lightgray'

    # Set node size
    node_size = 1000

    # Define the position for better visualization
    seed_value = 4
    
    pos = {f'O{t}': np.array([-0.64307049,  0.67121434]),
     f'G{t}': np.array([ 1., -0.16012511]),
     f'A{t}': np.array([-0.90599189,  0.15068216]),
     f'H{t}': np.array([ 0.50721255, -0.65769381]),
     f'E{t}': np.array([ 0.27184027, -0.07098749]),
     f'T{t}': np.array([ 0.05455484, -0.50582441]),
     f'J{t}': np.array([0.82960681, 0.39761469]),
     f'C{t}': np.array([-0.06881305,  0.60493769]),
     f'D{t}': np.array([-0.52304613, -0.20663306]),
     f'F{t}': np.array([-0.24173344, -0.51682124]),
     f'B{t}': np.array([-0.28055945,  0.29363624])}

#     pos = nx.spring_layout(G, seed=seed_value)

    # Increase plot size
    plt.figure(figsize=(10, 6))  # Adjust the width and height as desired

    # Iterate over edges and set edge color for nonzero weights
    edge_colors = []

    for pair in list(filtered_pairs):
        if edge_labels[pair] == 0.0:
            edge_colors.append('gray')
        else:
            edge_colors.append('orange') # Highlight nonzero edges

    # Draw the edges with customized colors and edge labels
    nx.draw_networkx_edges(G, pos, edgelist=list(filtered_pairs), edge_color=edge_colors)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=12)

    # Draw the nodes with customized color and size
    nx.draw_networkx_nodes(G, pos, node_color=node_color, node_size=node_size)

    # Draw the node labels
    nx.draw_networkx_labels(G, pos)

    # Plot adjustments
    plt.axis('off')
    plt.title(label=f"Delivery Flow of Truck {t}",
              loc="left",
              fontstyle='italic',
              fontweight=15,
              fontsize=15)

    # Display the graph
    plt.show()
#     print(f"graph_nodes_t{t}")
#     print(G.nodes)
    G.clear()
#     print(G.nodes)

<p style = 'font-size:16px;font-family:Arial'> The below table shows the total number of packages each truck will be delivering to different nodes along the path highlighted in the visual.</p>

In [ ]:
df_assignments[['Truck', 'Pkg']].groupby('Truck').count() 

In [ ]:
for t in t_ids:
    graph_viz(t) 

<p style = 'font-size:16px;font-family:Arial'>The above visualization shows the nodes and paths for all the 10 trucks. The bubbles show all the zones available and the edges shows the paths that can be traversed for the delivery of the packages. Based on the assignment of packages and the zones for each truck to be delivered the best route is assigned. The table above the visualization is for the total number of packages each truck is carrying. The orange colored lines show the best route of each truck and the number of packages the truck is carrying while travelling on each path. The path starts at the left most node of the orange line. The truck is carrying the number of packages mentioned on the first line. The number of packages dropped at the first node will be the subtraction of packages mentioned on the 1st and 2nd path, the number of packages dropped on the 2nd node will be subtraction of packages mentioned on 2nd and 3rd path and so on till all packages are dropped by the truck. Similarly, the paths and packages for all trucks are highlighted in orange.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>7. Conclusion.</b></p>

<p style = 'font-size:16px;font-family:Arial'>Thus, using the Mixed Integer Linear Programming(MLIP) function and calling that using the Script Table Operator(STO) we find the optimal route for the Trucks and the total cost involved in the package deliveries. </p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>8. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>

In [ ]:
tables = ['TRUCKS', 'PACKAGES','zone_links', 'actual_table']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    db_drop_table(table_name=table)    

In [ ]:
remove_context()

<footer style="padding-bottom:35px; background:#91A0AB; ">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023,2024. All Rights Reserved
        </div>
    </div>
</footer>